# Обучение SVM на PyTorch

SVM - это линейная модель со следующей функцией потерь:

$L(x, y) = \max(0, 1 - y (x^Tw + b)) + \frac{1}{2}\lambda \|w\|^2$

где $y$ это метка класса (-1 или 1), $x$ - вектор признаков, $w$ и $b$ - обучаемые параметры модели, а $\lambda$ - гиперпараметр регуляризации.

Предлагается использовать механизм автоматического дифференцирования PyTorch для оптимизации SVM методом градиентного спуска.

In [7]:
import torch
import sklearn.datasets

# Загрузим датасет бинарной классификации.
X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X = torch.tensor(X).float()
y = torch.tensor(y)
X = (X - X.mean(0, keepdim=True)) / X.std(0, keepdim=True)  # Нормализовать признаки.
print("Features:", X.shape)
print("Labels:", y.shape)
print("Num classes:", y.max().item() + 1)

N, D = X.shape
print("Num elements:", N)
print("Num features:", D)

y = y * 2 - 1  # Конвертировать метки из 0, 1 в -1, 1.
print("Sample features:", X.flatten()[:5])
print("Sample targets:", y[-10:])

Features: torch.Size([569, 30])
Labels: torch.Size([569])
Num classes: 2
Num elements: 569
Num features: 30
Sample features: tensor([ 1.0961, -2.0715,  1.2688,  0.9835,  1.5671])
Sample targets: tensor([ 1,  1,  1, -1, -1, -1, -1, -1, -1,  1])


**Задание 1.** Реализуйте SVM loss на PyTorch для батча данных. Предлагается усреднить значения лосса для всех элементов батча.

Обратите внимание на функции torch.clip, torch.square и torch.linalg.norm.

In [11]:
def svm_loss(X, y, w, b, l=0.001):
    """SVM Loss.
    
    Args:
        X: Входные признаки с размерностью (Batch, Dim).
        y: Входные метки с размерностью (Batch).
        w: Вектор весов с размерностью (Dim).
        b: Смещение (скаляр).
        l: Параметр регуляризации.
        
    Returns:
        Функция возвращает скаляр со значением лосса.
    """
    # Ваше решение здесь.
    reg = 0.5 * l * w.square().sum()  # Scalar.
    logits = X @ w + b  # (Batch).
    delta = (1 - y * logits).clip(min=0)  # (Batch).
    return (delta + reg).mean()  # Scalar.


# Проверки.
loss = svm_loss(X, y,
                w=torch.ones(D, requires_grad=True),
                b=torch.full([], 0.5, requires_grad=True))
assert abs(loss.item() - 15.126455307006836) < 1e-5, loss.item()
assert loss.requires_grad
del loss

**Задание 2** Создайте оптимизатор с шагом обучения 0.1.

Обратите внимание на функцию torch.optim.SGD.

In [12]:
def make_optimizer(w, b):
    """Возвращает оптимизатор для параметров w и b."""
    # Ваше решение здесь.
    return torch.optim.SGD([w, b], lr=0.1)


# Проверки.
optimizer = make_optimizer(w=torch.ones(D, requires_grad=True),
                           b=torch.full([], 0.5, requires_grad=True))
assert abs(optimizer.param_groups[0]["lr"] - 0.1) < 1e-5
assert len(optimizer.param_groups) == 1
assert sum(p.numel() for p in optimizer.param_groups[0]["params"]) == 31
del optimizer

**Задание 3**. Реализуйте шаг обучения модели.

Не забудьте обнулить градиенты перед backward.

Обратите внимание на функции optimizer.zero_grad и optimizer.step. Чтобы запустить автоматическое дифференцирование нужно выполнить метод backward для значения лосс-функции.

In [17]:
def train_step(optimizer, X, y, w, b, l=0.001):
    """Сделать шаг оптимизации."""
    # Ваш код здесь.
    optimizer.zero_grad()
    loss = svm_loss(X, y, w, b, l)
    loss.backward()
    optimizer.step()
    
# Проверки.
w = torch.ones(D, requires_grad=True)
b = torch.full([], 0.5, requires_grad=True)
w.grad = torch.full_like(w, -1)
optimizer = make_optimizer(w, b)
train_step(optimizer, X, y, w, b)
assert abs(w.grad[0].item() - 0.6581) < 1e-4, w.grad[0]
assert abs(b.grad.item() + 0.2320) < 1e-4, b.grad
assert abs(w[0].item() - 0.9342) < 1e-4, w[0]
assert abs(b.item() - 0.5232) < 1e-4, b
del w, b, optimizer

**Задание 4.** Реализуйте цикл обучения: сделайте 100 шагов из начальной точки.

In [19]:
def get_accuracy(X, y, w, b):
    logits = (X * w).sum(-1) + b
    predictions = (logits > 0).long() * 2 - 1
    return (y == predictions).float().mean().item()


def training_loop(X, y, w, b):
    # Ваш код здесь.
    optimizer = make_optimizer(w, b)
    for _ in range(1000):
        train_step(optimizer, X, y, w, b)


w = torch.arange(D).float()
w.requires_grad = True
b = torch.tensor([0.0], requires_grad=True)

print("Initial accuracy:", get_accuracy(X, y, w, b))
training_loop(X, y, w, b)
print("Final accuracy:", get_accuracy(X, y, w, b))

# Проверки.
assert get_accuracy(X, y, w, b) > 0.9

Initial accuracy: 0.12302284687757492
Final accuracy: 0.9156414866447449
